<a href="https://colab.research.google.com/github/sominshim/Predicting_Personality_through_Text/blob/Ujin/predict_personality_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MBTI(Myers-Briggs Type Indicator) RNN

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

In [ ]:
print("버전: ", tf.__version__)

버전:  2.3.0


### load dataset

In [ ]:
#드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/My Drive/mbti_1.csv'

In [ ]:
#pandas read_csv로 불러오기
import pandas as pd
data = pd.read_csv(filename)
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
text = data
print(text.shape)

print(text[0:5])
print(text.iloc[2])

(8675, 2)
   type                                              posts
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1  ENTP  'I'm finding the lack of me in these posts ver...
2  INTP  'Good one  _____   https://www.youtube.com/wat...
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...
4  ENTJ  'You're fired.|||That's another silly misconce...
type                                                  INTP
posts    'Good one  _____   https://www.youtube.com/wat...
Name: 2, dtype: object


### Preprocessing labels

mbti one hot encoding

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# One hot encode labels
labels=text.index.tolist()
encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
labels=encoder.fit_transform(labels)
labels=np.array(labels)
print(labels[50:55])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
mbti_dict={0:'ENFJ',1:'ENFP',2:'ENTJ',3:'ENTP',4:'ESFJ',5:'ESFP',6:'ESTJ',7:'ESTP',8:'INFJ',9:'INFP',10:'INTJ',11:'INTP',12:'ISFJ',13:'ISFP',14:'ISFP',15:'ISTP'}

### Preprocessing posts
We can see that the posts are very noisy, so they need to be cleaned. For this I'm doing the following:


Converting all letters to lowercase.

Remove '|||'

Removing punctuation.

Removing URLs, links etc..

Convert words to integers

We'll leave unicode emojis alone.

In [ ]:
import re

# Function to clean data ... will be useful later
def post_cleaner(post):
    """cleans individual posts`.
    Args:
        post-string
    Returns:
         cleaned up post`.
    """
    # Covert all uppercase characters to lower case
    post = post.lower() 
    
    # Remove |||
    post=post.replace('|||',"") 

    # Remove URLs, links etc
    post = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', post, flags=re.MULTILINE) 
    # This would have removed most of the links but probably not all 

    # Remove puntuations 
    puncs1=['@','#','$','%','^','&','*','(',')','-','_','+','=','{','}','[',']','|','\\','"',"'",';',':','<','>','/']
    for punc in puncs1:
        post=post.replace(punc,'') 

    puncs2=[',','.','?','!','\n']
    for punc in puncs2:
        post=post.replace(punc,' ') 
    # Remove extra white spaces
    post=re.sub( '\s+', ' ', post ).strip()
    return post

In [ ]:
# Clean up posts
# Covert pandas dataframe object to list. I prefer using lists for prepocessing. 
posts=text.posts.tolist()
posts=[post_cleaner(post) for post in posts]

In [ ]:
posts[0]

'and intj moments sportscenter not top ten plays prankswhat has been the most lifechanging experience in your life on repeat for most of today may the perc experience immerse you the last thing my infj friend posted on his facebook before committing suicide the next day rest in peace~ enfj7 sorry to hear of your distress its only natural for a relationship to not be perfection all the time in every moment of existence try to figure the hard times as times of growth as 84389 84390 welcome and stuff game set match prozac wellbrutin at least thirty minutes of moving your legs and i dont mean moving them while sitting in your same desk chair weed in moderation maybe try edibles as a healthier alternative basically come up with three items youve determined that each type or whichever types you want to do would more than likely use given each types cognitive functions and whatnot when left by all things in moderation sims is indeed a video game and a good one at that note a good one at that 

In [ ]:
# Count total words
from collections import Counter

word_count=Counter()
for post in posts:
    word_count.update(post.split(" "))

In [ ]:
# Size of the vocabulary available to the RNN
vocab_len=len(word_count)
print(vocab_len)

print(len(posts[0]))

172984
3094


### convert words to integers

In [ ]:
# Create a look up table 
vocab = sorted(word_count, key=word_count.get, reverse=True)
# Create your dictionary that maps vocab words to integers here
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

posts_ints=[]
for post in posts:
    posts_ints.append([vocab_to_int[word] for word in post.split()])

print(posts_ints[0])
print(len(posts_ints[0]))

[5, 141, 1287, 61293, 22, 703, 1850, 2069, 61294, 89, 72, 2, 84, 13390, 286, 11, 39, 108, 24, 2176, 14, 84, 6, 375, 196, 2, 723, 286, 12342, 7, 2, 241, 115, 12, 133, 148, 574, 24, 134, 1035, 185, 5881, 2140, 2, 459, 189, 762, 11, 61295, 61296, 279, 3, 416, 6, 39, 7396, 34, 86, 814, 14, 4, 238, 3, 22, 20, 3969, 43, 2, 59, 11, 217, 475, 6, 1558, 154, 3, 524, 2, 205, 242, 26, 242, 6, 2559, 26, 61297, 61298, 308, 5, 274, 492, 712, 1216, 12655, 61299, 41, 257, 7322, 987, 6, 1077, 39, 3300, 5, 1, 31, 161, 1077, 64, 169, 1108, 11, 39, 130, 2803, 3858, 1989, 11, 7024, 150, 154, 28515, 26, 4, 5882, 2233, 510, 211, 62, 16, 565, 3459, 443, 2804, 8, 309, 95, 32, 6508, 218, 7, 81, 3, 33, 49, 47, 90, 408, 221, 626, 309, 218, 630, 342, 5, 4236, 40, 532, 73, 43, 76, 11, 7024, 5185, 9, 1027, 4, 464, 492, 5, 4, 75, 46, 41, 8, 940, 4, 75, 46, 41, 8, 9, 833, 1489, 11, 8, 1, 56, 22, 354, 10262, 2, 684, 6, 111, 626, 11286, 613, 194, 35, 103, 39, 417, 464, 568, 1120, 62, 5, 35, 21, 39, 92, 954, 417, 464, 568

### Make posts uniform

We can see that the lengths of the posts aren't uniform, so we'll limit number of words in each post to 1000.For posts with less than 1000 words, we'll pad with zeros.

In [ ]:
posts_lens = Counter([len(x) for x in posts])
print("Zero-length reviews: {}".format(posts_lens[0]))
print("Maximum review length: {}".format(max(posts_lens)))
print("Minimum review length: {}".format(min(posts_lens)))

seq_len = 500
features=np.zeros((len(posts_ints),seq_len),dtype=int)
for i, row in enumerate(posts_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
print(features[:10])

Zero-length reviews: 0
Maximum review length: 9588
Minimum review length: 13
[[   5  141 1287 ...  222  278    1]
 [  18  751    2 ...    2 1660 4189]
 [  75   46  386 ...   24 2234   75]
 ...
 [   1  259    3 ...   17  631    3]
 [  18   22  120 ... 4330  659   11]
 [  11   19 1197 ...   47 2496  112]]


Preparing tranining, test and validation datasets

In [ ]:
# Split data into training, test and validation

split_frac = 0.8

num_ele=int(split_frac*len(features))
rem_ele=len(features)-num_ele
train_x, val_x = features[:num_ele],features[num_ele:int(rem_ele/2)+num_ele]
train_y, val_y = labels[:num_ele],labels[num_ele:int(rem_ele/2)+num_ele]

test_x = features[num_ele+int(rem_ele/2):]
test_y = labels[num_ele+int(rem_ele/2):]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(6940, 500) 
Validation set: 	(867, 500) 
Test set: 		(868, 500)


# RNN

In [ ]:
labels_

<tf.Tensor 'labels:0' shape=(None, None) dtype=int32>

In [ ]:
lstm_size = 256
lstm_layers = 1
batch_size = 256
learning_rate = 0.01
embed_dim=250

In [ ]:
n_words = len(vocab_to_int) + 1 # Adding 1 because we use 0's for padding, dictionary started at 1

# Create the graph object 새 그래프 생성
graph = tf.Graph()

# Add nodes to the graph 노드추가
with graph.as_default():
    input_data = tf.compat.v1.placeholder(tf.int32, [None, None], name='inputs')  # placeholder (dtype : 데이터 타입, shape : 입력 데이터의 형태상수or다차원배열, name : 해당 placeholder의 이름)
    labels_ = tf.compat.v1.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.compat.v1.placeholder(tf.float32, name='keep_prob')

In [ ]:
# Embedding
with graph.as_default():
    embedding= tf.Variable(tf.random.uniform(shape=(n_words,embed_dim),minval=-1,maxval=1))
    embed=tf.nn.embedding_lookup(embedding,input_data) #텐서 목록에서 병렬 조회 수행 (params:완전한 임베딩 텐서를 나타내는 단일 텐서 또는 "div"파티션 전략에 따라 분할 된 임베딩 텐서를 나타내는 첫 번째 차원을 제외하고 모두 동일한 형태의 텐서 목록입니다, ids : Tensor유형 int32또는 int64ID를 포함하는가에 조회 할 수 있습니다 params)
    print(embed.shape)

(None, None, 250)


In [ ]:
#pip install tensorflow-gpu==1.14

In [ ]:
# LSTM cell
with graph.as_default():
    # create basic LSTM cell
    lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size) #num_units : 훈련 시작시 망각의 규모를 줄이기 위해 망각 게이트의 편향에 forget_bias (기본값 : 1)를 추가
    
    # Add dropout to the cell
    drop = tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([drop]* lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [ ]:
with graph.as_default():
    outputs,final_state=tf.compat.v1.nn.dynamic_rnn(cell,embed,dtype=tf.float32 )

In [ ]:
tf.compat.v1.disable_eager_execution() # need to disable eager in TF2.x
# Build a graph.
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

# Launch the graph in a session.
sess = tf.compat.v1.Session()

# Evaluate the tensor `c`.
print(sess.run(c)) # prints 30.0

30.0


In [ ]:
labels_ = tf.compat.v1.placeholder(tf.int32, [No, None], name='labels')

In [ ]:
sess.run(labels_)

ValueError: ignored

In [ ]:
with graph.as_default():
    
    pre = tf.keras.layers.Dense(16,  activation = tf.nn.relu)(outputs[:,-1])
    predictions=tf.keras.layers.Dense(16, activation='softmax')(pre)
    
    cost = tf.compat.v1.losses.mean_squared_error(labels_,predictions)
    optimizer =  tf.keras.optimizers.Adam(learning_rate).minimize(cost)

ValueError: ignored

In [ ]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
def get_batches(x, y, batch_size=100):    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [ ]:
epochs = 3

with graph.as_default():
    saver = tf.compat.v1.train.Saver()

with tf.compat.v1.Session(graph=graph) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {input_data: x,
                    labels_: y,
                    keep_prob: 1.0,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {input_data: x,
                            labels_: y,
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/mbti.ckpt")

ValueError: ignored